Anne Chel, 10727477
Tyler Cools, 11004851
Kyle Molenaar, 10779027

# Fuzzy logic model for price classification of used cars

### Hierachie
- system 1: 
    * input: number of kilometers, age of the car.
    * output: price
- system 2:
    * input: gearbox, horsepower.
    * output: price
- system 3: 
    * input: fueltype, vehicletype.
    * output: price
- system 4:
    * input: combined output of system 1, 2 and 3, brand of the car.
    * output: price

The <b>fuzzifier</b> needs:
- input variables, output variables, described above. 
- membership functions for its variables, in this project three are used: triangular fuzzy logic membership functions, trapezoidal fuzzy logic membership function and singleton membership functions. 

The <b>rules</b> include:
- an antecedent containing a selected mf per variable (e.g. if x1 is low ...)
- operators: and, or, not (only consider one operator in a rule's antecedent)
- a consequent containing a selected mf for the output variable (Mamdani) or a formula for the output variable (TSK)

The <b>inference mechanism</b> does:
- uses the firing strength of a rule to calculate the result of the implication
- (for which it uses an implication operator: min, prod)
- aggregates the results

The aggregated result is then <b>defuzzified</b> and becomes a crisp value.


In [367]:
# Membership functions 

import math
import numpy as np
from collections import defaultdict, Counter
from __future__ import division

class TriangularMF:
    """Triangular fuzzy logic membership function class."""
    def __init__(self, name, start, top, end):
        self.name = name
        self.start = start
        self.top = top
        self.end = end

    def calculate_membership(self, x):
        if x <= self.start:
            return 0
        elif x >= self.start and x <= self.top:
            return (x-self.start)/(self.top-self.start)
        elif x >= self.top and x <= self.end:
            return (self.end-x)/(self.end-self.top)
        elif x >= self.end:
            return 0
        
class TrapezoidalMF:
    """Trapezoidal fuzzy logic membership function class."""
    def __init__(self, name, start, left_top, right_top, end):
        self.name = name
        self.start = start
        self.left_top = left_top
        self.right_top = right_top
        self.end = end

    def calculate_membership(self, x):
        if x <= self.start:
            return 0
        elif x >= self.start and x <= self.left_top:
            return (x-self.start)/(self.left_top-self.start)
        elif x >= self.left_top and x <= self.right_top:
            return 1
        elif x >= self.right_top and x <= self.end:
            return (self.end-x)/(self.end-self.right_top)
        elif x >= self.end:
            return 0

class SingletonMF:
    """Singleton fuzzy locgic membership function class."""
    def __init__(self, name, value):
        self.name = name
        self.value = value
    
    def calculate_membership(self, x):
        if x == self.value:
            return 1
        else:
            return 0

In [368]:
class Variable:
    """General class for variables in an FLS."""
    def __init__(self, name, range, mfs):
        self.name = name
        self.range = range
        self.mfs = mfs

    def calculate_memberships(self, x):
        """Test function to check whether
        you put together the right mfs in your variables."""
        return {
            mf.name : mf.calculate_membership(x)
            for mf in self.mfs
        }

    def get_mf_by_name(self, name):
        for mf in self.mfs:
            if mf.name == name:
                return mf

class Input(Variable):
    """Class for input variables, inherits 
    variables and functions from superclass Variable."""
    def __init__(self, name, range, mfs):
        super().__init__(name, range, mfs)
        self.type = "input"

class Output(Variable):
    """Class for output variables, inherits 
    variables and functions from superclass Variable."""
    def __init__(self, name, range, mfs):
        super().__init__(name, range, mfs)
        self.type = "output"

In [369]:
# Input variables for used car system.

# fuelType ['diesel', 'benzin', 'lpg', 'hybrid', 'elektro']
fuelType_mf1 = SingletonMF("diesel", 1)
fuelType_mf2 = SingletonMF("benzin", 2)
fuelType_mf3 = SingletonMF("lpg", 3)
fuelType_mf4 = SingletonMF("cng", 4)
mfs_fuelType = [fuelType_mf1, fuelType_mf2, fuelType_mf3, fuelType_mf4]
fuelType = Input("fuelType", (1, 4), mfs_fuelType)

# yearOfRegistration [min = 1910,  max = 2018]
yearOfRegistration_mf1 = TrapezoidalMF("really old", 1909, 1910, 1940, 1964)
yearOfRegistration_mf2 = TriangularMF("very old", 1964, 1968, 1971)
yearOfRegistration_mf3 = TriangularMF("old", 1971, 1975, 1978)
yearOfRegistration_mf4 = TriangularMF("bit old", 1978, 1982, 1985)
yearOfRegistration_mf5 = TriangularMF("average", 1985, 1989, 1992)
yearOfRegistration_mf6 = TriangularMF("bit new", 1992, 1996, 1999)
yearOfRegistration_mf7 = TriangularMF("new", 1999, 2003, 2006)
yearOfRegistration_mf8 = TriangularMF("very new", 2006, 2010, 2013)
yearOfRegistration_mf9 = TrapezoidalMF("really new", 2010, 2017, 2018, 2019)
mfs_yearOfRegistration = [yearOfRegistration_mf1, yearOfRegistration_mf2, yearOfRegistration_mf3, 
                          yearOfRegistration_mf4, yearOfRegistration_mf5, yearOfRegistration_mf6,
                          yearOfRegistration_mf7,yearOfRegistration_mf8, yearOfRegistration_mf9 ]
yearOfRegistration = Input("yearOfRegistration", (1910, 2018), mfs_yearOfRegistration)

# kilometer [min = 5000, max = 150000]
kilometer_mf1 = TriangularMF("really low", 0, 5000, 15000)
kilometer_mf2 = TriangularMF("very low", 15000, 20000, 30000)
kilometer_mf3 = TriangularMF("low", 30000, 35000, 45000)
kilometer_mf4 = TriangularMF("bit low", 45000, 50000, 60000)
kilometer_mf5 = TriangularMF("average", 60000, 65000, 75000)
kilometer_mf6 = TriangularMF("bit high", 75000, 850000, 90000)
kilometer_mf7 = TriangularMF("high", 90000, 100000, 105000)
kilometer_mf8 = TriangularMF("very high", 120000, 130000, 135000)
kilometer_mf9 = TriangularMF("really high", 135000, 140000, 150000)
kilometer_mf10 = TriangularMF("highest", 100000, 120000, 150000)
mfs_kilometer = [kilometer_mf1, kilometer_mf2, kilometer_mf3, 
                kilometer_mf4, kilometer_mf5, kilometer_mf6,
                kilometer_mf7, kilometer_mf8, kilometer_mf9, 
                kilometer_mf10]
kilometer = Input("kilometer", (5000, 150000), mfs_kilometer)

# gearbox ['automatik', 'manuell']
gearbox_mf1 = SingletonMF("auto", 1)
gearbox_mf2 = SingletonMF("man", 2)
mfs_gearbox = [gearbox_mf1, gearbox_mf2]
gearbox = Input("gearbox", (1, 2), mfs_gearbox)

# vehicleType ['suv', 'kleinwagen', 'limousine', 'cabrio', 'bus', 'combi', 'coupe', 'andere']
vehicleType_mf4 = SingletonMF("suv", 4)
vehicleType_mf1 = SingletonMF("kleinwagen", 1)
vehicleType_mf5 = SingletonMF("limousin", 5)
vehicleType_mf3 = SingletonMF("bus", 3)
vehicleType_mf2 = SingletonMF("combi", 2)
mfs_vehicleType = [vehicleType_mf1, vehicleType_mf2, vehicleType_mf3, vehicleType_mf4, vehicleType_mf5]
vehicleType = Input("vehicleType", (1, 5), mfs_vehicleType)

# horsePower [min = 21, max = 776]
horsePower_mf1 = TrapezoidalMF("ultra low", 20, 21, 30, 50)
horsePower_mf2 = TriangularMF("low", 50, 90, 100)
horsePower_mf3 = TriangularMF("bit low", 100, 125, 150)
horsePower_mf4 = TriangularMF("medium", 150, 175, 200)
horsePower_mf5 = TriangularMF("bit high", 200, 225, 250)
horsePower_mf6 = TriangularMF("high", 250, 275, 300)
horsePower_mf7 = TriangularMF("ultra high", 300, 325, 350)
horsePower_mf8 = TrapezoidalMF("highest", 350, 500, 776, 777)
mfs_horsePower = [horsePower_mf1, horsePower_mf2, horsePower_mf3, horsePower_mf4, horsePower_mf5,
                 horsePower_mf6, horsePower_mf7, horsePower_mf8]
horsePower = Input("horsePower", (21, 776), mfs_horsePower)

# similar cars are grouped together
brand_mf1 = SingletonMF("cat1", 1)
brand_mf2 = SingletonMF("cat2", 2)
brand_mf3 = SingletonMF("cat3", 3)
brand_mf4 = SingletonMF("cat4", 4)
brand_mf5 = SingletonMF("cat5", 5)
brand_mf6 = SingletonMF("cat6", 6)
brand_mf7 = SingletonMF("cat7", 7)
brand_mf8 = SingletonMF("cat8", 8)

mfs_brand = [brand_mf1, brand_mf2, brand_mf3, brand_mf4, brand_mf5, brand_mf6, brand_mf7, brand_mf8]
brand = Input("brand", (1, 8), mfs_brand)

all_price_mf1 = SingletonMF("ultra cheap", 1)
all_price_mf2 = SingletonMF("bit cheap", 2)
all_price_mf3 = SingletonMF("cheap", 3)
all_price_mf4 = SingletonMF("medium", 4)
all_price_mf5 = SingletonMF("bit expensive", 5)
all_price_mf6 = SingletonMF("expensive", 6)
all_price_mf7 = SingletonMF("ultra expensive", 7)

mfs_all_price = [all_price_mf1, all_price_mf2, all_price_mf3, all_price_mf4, 
                 all_price_mf5, all_price_mf6, all_price_mf7]
allPrices = Input("allPrices", (1, 7), mfs_all_price)

In [370]:
# Output variable for used car system.

# price [min = 101, max = 249000]
price_mf1 = TriangularMF("ultra cheap", 0, 101, 1000)
price_mf2 = TriangularMF("bit cheap", 1000, 1500, 3000)
price_mf3 = TriangularMF("cheap", 3000, 5000, 5000)
price_mf4 = TriangularMF("medium", 5000, 8000, 10000)
price_mf5 = TriangularMF("bit expensive", 10000, 40000 , 50000)
price_mf6 = TriangularMF("expensive", 50000, 70000, 110000)
price_mf7 = TriangularMF("ultra expensive", 110000, 230000, 249000)
mfs_price = [price_mf1, price_mf2, price_mf3, price_mf4, price_mf5, price_mf6, price_mf7]
price = Output("price", (101, 249000), mfs_price)

inputs1 = [kilometer, yearOfRegistration]
inputs2 = [gearbox, horsePower]
inputs3 = [fuelType, vehicleType]
output = price

In [371]:
# Determine firing strength of all rules

class Rule:
    """Fuzzy rule class, initialized with an antecedent (list of strings),
    operator (string) and consequent (string)."""
    def __init__(self, n, antecedent, operator, consequent):
        self.number = n
        self.antecedent = antecedent
        self.operator = operator
        self.consequent = consequent
        self.firing_strength = 0

    def calculate_firing_strength(self, datapoint, inputs):
        sets = []
        for n in range(len(inputs)):
            x = datapoint[n]
            mfName = self.antecedent[n]
            mf = inputs[n].get_mf_by_name(mfName)
            fuzzySet = mf.calculate_membership(x)
            sets.append(fuzzySet)
        if self.operator == "and":
            self.firing_strength = min(sets)
        elif self.operator == "or":
            self.firing_strength = max(sets)
        return self.firing_strength

In [372]:
# Return largest firing strengths encountered

from collections import Counter

class Rulebase:
    """The fuzzy rulebase collects all rules for the FLS, can
    calculate the firing strengths of its rules."""
    def __init__(self, rules):
        self.rules = rules

    def calculate_firing_strengths(self, datapoint, inputs):
        result = Counter()
        for i, rule in enumerate(self.rules):
            fs = rule.calculate_firing_strength(datapoint, inputs)
            consequent = rule.consequent
            if fs > result[consequent]:
                result[consequent] = fs
        return result

In [373]:
# Rules

# year/km
rules_1 = [Rule(1, ["really low", "really old"], "and", "bit cheap"), 
           Rule(2, ["really low", "very old"], "and", "bit cheap"),
           Rule(3, ["really low", "old"], "and", "bit cheap"), 
           Rule(4, ["really low", "bit old"], "and", "medium"),
           Rule(5, ["really low", "average"], "and", "medium"), 
           Rule(6, ["really low", "bit new"], "and", "bit expensive"),
           Rule(7, ["really low", "new"], "and", "bit expensive"), 
           Rule(8, ["really low", "very new"], "and", "expensive"), 
           Rule(9, ["really low", "really new"], "and", "ultra expensive"),
           Rule(10, ["very low", "really old"], "and", "cheap"), 
           Rule(11, ["very low", "very old"], "and", "bit cheap"),
           Rule(12, ["very low", "old"], "and", "bit cheap"), 
           Rule(13, ["very low", "bit old"], "and", "bit cheap"),
           Rule(14, ["very low", "average"], "and", "medium"), 
           Rule(15, ["very low", "bit new"], "and", "mdcium"),
           Rule(16, ["very low", "new"], "and", "bit expensive"), 
           Rule(17, ["very low", "very new"], "and", "bit expensive"), 
           Rule(18, ["very low", "really new"], "and", "expensive"),
           Rule(19, ["low", "really old"], "and", "cheap"), 
           Rule(20, ["low", "very old"], "and", "bit cheap"),
           Rule(21, ["low", "old"], "and", "bit cheap"), 
           Rule(22, ["low", "bit old"], "and", "bit cheap"),
           Rule(23, ["low", "average"], "and", "medium"), 
           Rule(24, ["low", "bit new"], "and", "medium"),
           Rule(25, ["low", "new"], "and", "medium"), 
           Rule(26, ["low", "very new"], "and", "bit expensive"), 
           Rule(27, ["low", "really new"], "and", "bit expensive"),
           Rule(28, ["bit low", "really old"], "and", "cheap"), 
           Rule(29, ["bit low", "very old"], "and", "cheap"),
           Rule(30, ["bit low", "old"], "and", "bit cheap"), 
           Rule(31, ["bit low", "bit old"], "and", "bit cheap"),
           Rule(32, ["bit low", "average"], "and", "medium"), 
           Rule(33, ["bit low", "bit new"], "and", "medium"),
           Rule(34, ["bit low", "new"], "and", "medium"), 
           Rule(35, ["bit low", "very new"], "and", "medium"), 
           Rule(36, ["bit low", "really new"], "and", "bit expensive"),
           Rule(37, ["average", "really old"], "and", "cheap"), 
           Rule(38, ["average", "very old"], "and", "cheap"),
           Rule(39, ["average", "old"], "and", "cheap"), 
           Rule(40, ["average", "bit old"], "and", "bit cheap"),
           Rule(41, ["average", "average"], "and", "bit cheap"), 
           Rule(42, ["average", "bit new"], "and", "medium"),
           Rule(43, ["average", "new"], "and", "medium"), 
           Rule(44, ["average", "very new"], "and", "medium"), 
           Rule(45, ["average", "really new"], "and", "medium"),
           Rule(46, ["bit high", "really old"], "and", "ultra cheap"), 
           Rule(47, ["bit high", "very old"], "and", "cheap"),
           Rule(48, ["bit high", "old"], "and", "cheap"), 
           Rule(49, ["bit high", "bit old"], "and", "cheap"),
           Rule(50, ["bit high", "average"], "and", "cheap"), 
           Rule(51, ["bit high", "bit new"], "and", "bit cheap"),
           Rule(52, ["bit high", "new"], "and", "bit cheap"), 
           Rule(53, ["bit high", "very new"], "and", "bit cheap"), 
           Rule(54, ["bit high", "really new"], "and", "medium"),
           Rule(55, ["high", "really old"], "and", "ultra cheap"), 
           Rule(56, ["high", "very old"], "and", "ultra cheap"),
           Rule(57, ["high", "old"], "and", "ultra cheap"), 
           Rule(58, ["high", "bit old"], "and", "cheap"),
           Rule(59, ["high", "average"], "and", "cheap"), 
           Rule(60, ["high", "bit new"], "and", "cheap"),
           Rule(61, ["high", "new"], "and", "bit cheap"), 
           Rule(62, ["high", "very new"], "and", "bit cheap"), 
           Rule(63, ["high", "really new"], "and", "bit cheap"),        
           Rule(64, ["very high", "really old"], "and", "ultra cheap"), 
           Rule(65, ["very high", "very old"], "and", "ultra cheap"),
           Rule(66, ["very high", "old"], "and", "ultra cheap"), 
           Rule(67, ["very high", "bit old"], "and", "ultra cheap"),
           Rule(68, ["very high", "average"], "and", "cheap"), 
           Rule(69, ["very high", "bit new"], "and", "cheap"),
           Rule(70, ["very high", "new"], "and", "cheap"), 
           Rule(71, ["very high", "very new"], "and", "bit cheap"), 
           Rule(72, ["very high", "really new"], "and", "bit cheap"),
           Rule(73, ["really high", "really old"], "and", "ultra cheap"), 
           Rule(74, ["really high", "very old"], "and", "ultra cheap"),
           Rule(75, ["really high", "old"], "and", "ultra cheap"), 
           Rule(76, ["really high", "bit old"], "and", "ultra cheap"),
           Rule(77, ["really high", "average"], "and", "ultra cheap"), 
           Rule(78, ["really high", "bit new"], "and", "cheap"),
           Rule(79, ["really high", "new"], "and", "cheap"), 
           Rule(80, ["really high", "very new"], "and", "cheap"), 
           Rule(81, ["really high", "really new"], "and", "bit cheap"), 
           Rule(82, ["highest", "really old"], "and", "ultra cheap"), 
           Rule(83, ["highest", "very old"], "and", "ultra cheap"),
           Rule(84, ["highest", "old"], "and", "ultra cheap"), 
           Rule(85, ["highest", "bit old"], "and", "ultra cheap"),
           Rule(86, ["highest", "average"], "and", "ultra cheap"), 
           Rule(87, ["highest", "bit new"], "and", "ultra cheap"),
           Rule(88, ["highest", "new"], "and", "cheap"), 
           Rule(89, ["highest", "very new"], "and", "cheap"), 
           Rule(90, ["highest", "really new"], "and", "cheap")  
          ]

# gearbox/horsepower
rules_2 = [Rule(1, ["auto", "ultra low"], "and", "ultra cheap"), 
           Rule(2, ["auto", "low"], "and", "cheap"),
           Rule(3, ["auto", "bit low"], "and", "medium"), 
           Rule(4, ["auto", "medium"], "and", "medium"),
           Rule(5, ["auto", "bit high"], "and", "bit expensive"), 
           Rule(6, ["auto", "high"], "and", "expensive"),
           Rule(7, ["auto", "ultra high"], "and", "expensive"), 
           Rule(8, ["auto", "highest"], "and", "ultra expensive"), 
           Rule(9, ["man", "ultra low"], "and", "ultra cheap"), 
           Rule(10, ["man", "low"], "and", "ultra cheap"),
           Rule(11, ["man", "bit low"], "and", "bit cheap"), 
           Rule(12, ["man", "medium"], "and", "medium"),
           Rule(13, ["man", "bit high"], "and", "medium"), 
           Rule(14, ["man", "high"], "and", "bit expensive"),
           Rule(15, ["man", "ultra high"], "and", "bit expensive"), 
           Rule(16, ["man", "highest"], "and", "expensive")
          ]

# fuel/vehicle-type
rules_3 = [ Rule(1, ["diesel", "kleinwagen"], "and", "bit cheap"), 
            Rule(2, ["diesel", "combi"], "and", "medium"),
            Rule(3, ["diesel", "bus"], "and", "medium"), 
            Rule(4, ["diesel", "suv"], "and", "bit expensive"),
            Rule(5, ["diesel", "limousin"], "and", "bit expensive"), 
            Rule(6, ["lpg", "kleinwagen"], "and", "ultra cheap"),
            Rule(7, ["lpg", "combi"], "and", "cheap"), 
            Rule(8, ["lpg", "bus"], "and", "cheap"), 
            Rule(9, ["lpg", "suv"], "and", "cheap"),
            Rule(10, ["lpg", "limousin"], "and", "cheap"), 
            Rule(11, ["cng", "kleinwagen"], "and", "ultra cheap"), 
            Rule(12, ["cng", "combi"], "and", "ultra cheap"),
            Rule(13, ["cng", "bus"], "and", "ultra cheap"), 
            Rule(14, ["cng", "suv"], "and", "cheap"),
            Rule(15, ["cng", "limousin"], "and", "cheap"), 
            Rule(16, ["benzin", "kleinwagen"], "and", "medium"),
            Rule(17, ["benzin", "combi"], "and", "bit expensive"), 
            Rule(18, ["benzin", "bus"], "and", "bit expensive"), 
            Rule(19, ["benzin", "suv"], "and", "expensive"),
            Rule(20, ["benzin", "limousin"], "and", "expensive")   
          ]

# price1/price2/price3-categorie
rules_4 = [Rule(1, ["ultra cheap", "ultra cheap", "ultra cheap"], "and", "ultra cheap"),
           Rule(2, ["ultra cheap", "ultra cheap", "cheap"], "and", "ultra cheap"),
           Rule(3, ["ultra cheap", "ultra cheap", "bit cheap"], "and", "cheap"), 
           Rule(4, ["ultra cheap", "ultra cheap", "medium"], "and", "cheap"),
           Rule(5, ["ultra cheap", "ultra cheap", "bit expensive"], "and", "bit cheap"),
           Rule(6, ["ultra cheap", "ultra cheap", "expensive"], "and", "medium"),
           Rule(7, ["ultra cheap", "ultra cheap", "ultra expensive"], "and", "medium"),
           Rule(8, ["cheap", "cheap", "ultra cheap"], "and", "cheap"),
           Rule(9, ["cheap", "cheap", "cheap"], "and", "cheap"),
           Rule(10, ["cheap", "cheap", "bit cheap"], "and", "cheap"), 
           Rule(11, ["cheap", "cheap", "medium"], "and", "bit cheap"),
           Rule(12, ["cheap", "cheap", "bit expensive"], "and", "bit cheap"),
           Rule(13, ["cheap", "cheap", "expensive"], "and", "medium"),
           Rule(14, ["cheap", "cheap", "ultra expensive"], "and", "medium"),
           Rule(15, ["bit cheap", "bit cheap", "ultra cheap"], "and", "cheap"),
           Rule(16, ["bit cheap", "bit cheap", "cheap"], "and", "bit cheap"),
           Rule(17, ["bit cheap", "bit cheap", "bit cheap"], "and", "bit cheap"), 
           Rule(18, ["bit cheap", "bit cheap", "medium"], "and", "bit cheap"),
           Rule(19, ["bit cheap", "bit cheap", "bit expensive"], "and", "medium"),
           Rule(20, ["bit cheap", "bit cheap", "expensive"], "and", "medium"),
           Rule(21, ["bit cheap", "bit cheap", "ultra expensive"], "and", "bit expensive"),    
           Rule(22, ["medium", "medium", "ultra cheap"], "and", "bit cheap"),
           Rule(23, ["medium", "medium", "cheap"], "and", "bit cheap"),
           Rule(24, ["medium", "medium", "bit cheap"], "and", "medium"), 
           Rule(25, ["medium", "medium", "medium"], "and", "medium"),
           Rule(26, ["medium", "medium", "bit expensive"], "and", "medium"),
           Rule(27, ["medium", "medium", "expensive"], "and", "bit expensive"),
           Rule(28, ["medium", "medium", "ultra expensive"], "and", "bit expensive"),
           Rule(29, ["bit expensive", "bit expensive", "ultra cheap"], "and", "medium"),
           Rule(30, ["bit expensive", "bit expensive", "cheap"], "and", "medium"),
           Rule(31, ["bit expensive", "bit expensive", "bit cheap"], "and", "medium"), 
           Rule(32, ["bit expensive", "bit expensive", "medium"], "and", "bit expensive"),
           Rule(33, ["bit expensive", "bit expensive", "bit expensive"], "and", "bit expensive"),
           Rule(34, ["bit expensive", "bit expensive", "expensive"], "and", "bit expensive"),
           Rule(35, ["bit expensive", "bit expensive", "ultra expensive"], "and", "expensive"),     
           Rule(36, ["expensive", "expensive", "ultra cheap"], "and", "medium"),
           Rule(37, ["expensive", "expensive", "cheap"], "and", "medium"),
           Rule(38, ["expensive", "expensive", "bit cheap"], "and", "bit expensive"), 
           Rule(39, ["expensive", "expensive", "medium"], "and", "bit expensive"),
           Rule(40, ["expensive", "expensive", "bit expensive"], "and", "expensive"),
           Rule(41, ["expensive", "expensive", "expensive"], "and", "expensive"),
           Rule(41, ["expensive", "expensive", "ultra expensive"], "and", "expensive"),          
           Rule(42, ["ultra expensive", "ultra expensive", "ultra cheap"], "and", "bit expensive"),
           Rule(43, ["ultra expensive", "ultra expensive", "cheap"], "and", "bit expensive"),
           Rule(44, ["ultra expensive", "ultra expensive", "bit cheap"], "and", "expensive"), 
           Rule(45, ["ultra expensive", "ultra expensive", "medium"], "and", "expensive"),
           Rule(46, ["ultra expensive", "ultra expensive", "bit expensive"], "and", "expensive"),
           Rule(47, ["ultra expensive", "ultra expensive", "expensive"], "and", "ultra expensive"),
           Rule(48, ["ultra expensive", "ultra expensive", "ultra expensive"], "and", "ultra expensive"),
           Rule(49, ["ultra cheap", "cheap", "ultra cheap"], "and", "ultra cheap"),
           Rule(50, ["ultra cheap", "bit cheap", "ultra cheap"], "and", "cheap"),
           Rule(51, ["ultra cheap", "medium", "ultra cheap"], "and", "cheap"), 
           Rule(52, ["ultra cheap", "bit expensive", "ultra cheap"], "and", "bit cheap"),
           Rule(53, ["ultra cheap", "expensive", "ultra cheap"], "and", "medium"),
           Rule(54, ["ultra cheap", "ultra expensive", "ultra cheap"], "and", "medium"),         
           Rule(55, ["cheap", "ultra cheap", "cheap"], "and", "cheap"),
           Rule(56, ["cheap", "bit cheap", "cheap"], "and", "cheap"), 
           Rule(57, ["cheap", "medium", "cheap"], "and", "bit cheap"),
           Rule(58, ["cheap", "bit expensive", "cheap"], "and", "bit cheap"),
           Rule(59, ["cheap", "expensive", "cheap"], "and", "medium"),
           Rule(60, ["cheap", "ultra expensive", "cheap"], "and", "medium"),           
           Rule(61, ["medium", "ultra cheap", "medium"], "and", "bit cheap"),
           Rule(62, ["medium",  "cheap", "medium"], "and", "bit cheap"),
           Rule(63, ["medium", "bit cheap", "medium"], "and", "medium"), 
           Rule(64, ["medium", "bit expensive", "medium"], "and", "medium"),
           Rule(65, ["medium",  "expensive", "medium"], "and", "bit expensive"),
           Rule(66, ["medium", "ultra expensive", "medium"], "and", "bit expensive"),   
           Rule(67, ["bit expensive", "ultra cheap", "bit expensive"], "and", "medium"),
           Rule(68, ["bit expensive", "cheap", "bit expensive"], "and", "medium"),
           Rule(69, ["bit expensive", "bit cheap", "bit expensive"], "and", "medium"), 
           Rule(70, ["bit expensive",  "medium", "bit expensive"], "and", "bit expensive"),
           Rule(71, ["bit expensive",  "expensive", "bit expensive"], "and", "bit expensive"),
           Rule(71, ["bit expensive",  "ultra expensive", "bit expensive"], "and", "expensive"),           
           Rule(72, ["expensive", "ultra cheap", "expensive"], "and", "medium"),
           Rule(73, ["expensive",  "cheap", "expensive"], "and", "medium"),
           Rule(74, ["expensive",  "bit cheap", "expensive"], "and", "bit expensive"), 
           Rule(75, ["expensive",  "medium", "expensive"], "and", "bit expensive"),
           Rule(76, ["expensive",  "bit expensive", "expensive"], "and", "expensive"),
           Rule(77, ["expensive", "ultra expensive", "expensive"], "and", "expensive"),
           Rule(78, ["ultra expensive", "ultra cheap", "ultra expensive"], "and", "bit expensive"),
           Rule(79, ["ultra expensive", "cheap", "ultra expensive"], "and", "bit expensive"),
           Rule(80, ["ultra expensive",  "bit cheap", "ultra expensive"], "and", "expensive"), 
           Rule(81, ["ultra expensive",  "medium", "ultra expensive"], "and", "expensive"),
           Rule(82, ["ultra expensive",  "bit expensive", "ultra expensive"], "and", "expensive"),
           Rule(83, ["ultra expensive",  "expensive", "ultra expensive"], "and", "ultra expensive"), 
           Rule(84, ["cheap", "ultra cheap", "ultra cheap"], "and", "ultra cheap"),
           Rule(85, ["bit cheap", "ultra cheap", "ultra cheap"], "and", "cheap"),
           Rule(86, [ "medium", "ultra cheap", "ultra cheap"], "and", "cheap"), 
           Rule(87, [ "bit expensive", "ultra cheap", "ultra cheap"], "and", "bit cheap"),
           Rule(88, ["expensive", "ultra cheap", "ultra cheap"], "and", "medium"),
           Rule(89, [ "ultra expensive", "ultra cheap", "ultra cheap"], "and", "medium"), 
           Rule(90, ["ultra cheap", "cheap", "cheap"], "and", "cheap"),
           Rule(91, ["bit cheap", "cheap", "cheap"], "and", "cheap"), 
           Rule(92, [ "medium", "cheap", "cheap"], "and", "bit cheap"),
           Rule(93, ["bit expensive", "cheap", "cheap"], "and", "bit cheap"),
           Rule(94, ["expensive", "cheap", "cheap"], "and", "medium"),
           Rule(95, ["ultra expensive", "cheap", "cheap"], "and", "medium"),
           Rule(96, [ "ultra cheap", "medium", "medium"], "and", "bit cheap"),
           Rule(97, [ "cheap", "medium", "medium"], "and", "bit cheap"),
           Rule(98, ["bit cheap", "medium", "medium"], "and", "medium"), 
           Rule(99, ["bit expensive", "medium", "medium"], "and", "medium"),
           Rule(100, [ "expensive", "medium", "medium"], "and", "bit expensive"),
           Rule(101, [ "ultra expensive", "medium", "medium"], "and", "bit expensive"),           
           Rule(102, ["ultra cheap", "bit expensive", "bit expensive"], "and", "medium"),
           Rule(103, ["cheap", "bit expensive", "bit expensive"], "and", "medium"),
           Rule(104, ["bit cheap", "bit expensive", "bit expensive"], "and", "medium"), 
           Rule(105, ["medium", "bit expensive", "bit expensive"], "and", "bit expensive"),
           Rule(106, ["expensive", "bit expensive", "bit expensive"], "and", "bit expensive"),
           Rule(107, ["ultra expensive", "bit expensive", "bit expensive"], "and", "expensive"),           
           Rule(108, ["ultra cheap", "expensive", "expensive"], "and", "medium"),
           Rule(109, ["cheap", "expensive", "expensive"], "and", "medium"),
           Rule(110, ["bit cheap", "expensive", "expensive"], "and", "bit expensive"), 
           Rule(111, ["medium", "expensive", "expensive"], "and", "bit expensive"),
           Rule(112, ["bit expensive", "expensive", "expensive"], "and", "expensive"),
           Rule(113, ["ultra expensive", "expensive", "expensive"], "and", "expensive"),
           Rule(114, ["ultra cheap", "ultra expensive", "ultra expensive"], "and", "bit expensive"),
           Rule(115, ["cheap", "ultra expensive", "ultra expensive"], "and", "bit expensive"),
           Rule(116, ["bit cheap", "ultra expensive", "ultra expensive"], "and", "expensive"), 
           Rule(117, [ "medium", "ultra expensive", "ultra expensive"], "and", "expensive"),
           Rule(118, ["bit expensive", "ultra expensive", "ultra expensive"], "and", "expensive"),
           Rule(119, ["expensive", "ultra expensive", "ultra expensive"], "and", "ultra expensive"),
         ]

# brand/price categorie
rules_5 = [Rule(1, ["cat1", "ultra cheap"], "and", "ultra cheap"), 
           Rule(2, ["cat1", "cheap"], "and", "ultra cheap"),
           Rule(3, ["cat1", "bit cheap"], "and", "cheap"), 
           Rule(4, ["cat1", "medium"], "and", "bit cheap"),
           Rule(5, ["cat1", "bit expensive"], "and", "medium"), 
           Rule(6, ["cat1", "expensive"], "and", "bit expensive"),
           Rule(7, ["cat1", "ultra expensive"], "and", "expensive"), 
           Rule(8, ["cat2", "ultra cheap"], "and", "ultra cheap"), 
           Rule(9, ["cat2", "cheap"], "and", "cheap"),
           Rule(10, ["cat2", "bit cheap"], "and", "bit cheap"), 
           Rule(11, ["cat2", "medium"], "and", "medium"),
           Rule(12, ["cat2", "bit expensive"], "and", "bit expensive"), 
           Rule(13, ["cat2", "expensive"], "and", "expensive"),
           Rule(14, ["cat2", "ultra expensive"], "and", "ultra expensive"),            
           Rule(15, ["cat3", "ultra cheap"], "and", "cheap"), 
           Rule(16, ["cat3", "cheap"], "and", "cheap"),
           Rule(17, ["cat3", "bit cheap"], "and", "bit cheap"), 
           Rule(18, ["cat3", "medium"], "and", "bit cheap"),
           Rule(19, ["cat3", "bit expensive"], "and", "medium"), 
           Rule(20, ["cat3", "expensive"], "and", "medium"),
           Rule(21, ["cat3", "ultra expensive"], "and", "bit expensive"),           
           Rule(22, ["cat4", "ultra cheap"], "and", "cheap"), 
           Rule(23, ["cat4", "cheap"], "and", "bit cheap"),
           Rule(24, ["cat4", "bit cheap"], "and", "bit cheap"), 
           Rule(25, ["cat4", "medium"], "and", "medium"),
           Rule(26, ["cat4", "bit expensive"], "and", "medium"), 
           Rule(27, ["cat4", "expensive"], "and", "medium"),
           Rule(28, ["cat4", "ultra expensive"], "and", "bit expensive"), 
           Rule(29, ["cat5", "ultra cheap"], "and", "bit cheap"), 
           Rule(30, ["cat5", "cheap"], "and", "bit cheap"),
           Rule(31, ["cat5", "bit cheap"], "and", "medium"), 
           Rule(32, ["cat5", "medium"], "and", "medium"),
           Rule(33, ["cat5", "bit expensive"], "and", "medium"), 
           Rule(34, ["cat5", "expensive"], "and", "bit expensive"),
           Rule(35, ["cat5", "ultra expensive"], "and", "bit expensive"), 
           Rule(36, ["cat6", "ultra cheap"], "and", "bit cheap"), 
           Rule(37, ["cat6", "cheap"], "and", "medium"),
           Rule(38, ["cat6", "bit cheap"], "and", "medium"), 
           Rule(39, ["cat6", "medium"], "and", "medium"),
           Rule(40, ["cat6", "bit expensive"], "and", "bit expensive"), 
           Rule(41, ["cat6", "expensive"], "and", "bit expensive"),
           Rule(42, ["cat6", "ultra expensive"], "and", "expensive"),            
           Rule(43, ["cat7", "ultra cheap"], "and", "medium"), 
           Rule(44, ["cat7", "cheap"], "and", "medium"),
           Rule(45, ["cat7", "bit cheap"], "and", "medium"), 
           Rule(46, ["cat7", "medium"], "and", "bit expensive"),
           Rule(47, ["cat7", "bit expensive"], "and", "bit expensive"), 
           Rule(48, ["cat7", "expensive"], "and", "expensive"),
           Rule(49, ["cat7", "ultra expensive"], "and", "expensive"), 
           Rule(50, ["cat8", "ultra cheap"], "and", "medium"), 
           Rule(51, ["cat8", "cheap"], "and", "medium"),
           Rule(52, ["cat8", "bit cheap"], "and", "bit expensive"), 
           Rule(53, ["cat8", "medium"], "and", "bit expensive"),
           Rule(54, ["cat8", "bit expensive"], "and", "expensive"), 
           Rule(55, ["cat8", "expensive"], "and", "expensive"),
           Rule(56, ["cat8", "ultra expensive"], "and", "ultra expensive")       
        ]

In [374]:
# Combine three input rules to one ouput, and use this as input.

# price/km
datapoint = [2500, 1960]
rulebase = Rulebase(rules_1)
prijs1 = rulebase.calculate_firing_strengths(datapoint, inputs1)

# gearbox/horsepower
datapoint = [1, 110]
rulebase = Rulebase(rules_2)
prijs2 = rulebase.calculate_firing_strengths(datapoint, inputs2)

# fuel/vehicle-type
datapoint = [1, 3]
rulebase = Rulebase(rules_3)
prijs3 = rulebase.calculate_firing_strengths(datapoint, inputs3)

# ugly look-up for collecting consequent for further computation
from collections import *

class OrderedCounter(Counter, OrderedDict):
    pass

consequents = []
for e in prijs1, prijs2, prijs3:
    counterlist = OrderedCounter(e)
    for e in counterlist.keys():
        if e == 'ultra cheap':
            e = 1
        if e == 'cheap':
            e = 2        
        if e == 'bit cheap':
            e = 3
        if e == 'medium':
            e = 4
        if e == 'bit expensive':
            e = 5            
        if e == 'expensive':
            e = 6
        if e == 'ultra expensive':
            e = 7
        consequents.append(e)

# combine previous three systems to one new input
datapoints = [consequents[0], consequents[1], consequents[2]]
input4= [allPrices, allPrices, allPrices]
rulebase = Rulebase(rules_4)
prijs4 = rulebase.calculate_firing_strengths(datapoints, input4)

# get correct consequent
counterlist = OrderedCounter(prijs4)
q=0
for e in counterlist.keys():
        if e == 'ultra cheap':
            q = 1
        if e == 'cheap':
            q = 2        
        if e == 'bit cheap':
            q = 3
        if e == 'medium':
            q = 4
        if e == 'bit expensive':
            q = 5            
        if e == 'expensive':
            q = 6
        if e == 'ultra expensive':
            q = 7


In [375]:
class Reasoner:
    def __init__(self, rulebase, inputs, output, n_points, defuzzification):
        self.rulebase = rulebase
        self.inputs = inputs
        self.output = output
        self.discretize = n_points
        self.defuzzification = defuzzification

    def inference(self, datapoint):
        # 1. Calculate the highest firing strength found in the rules per 
        # membership function of the output variable
        
        strength = rulebase.calculate_firing_strengths(datapoint, inputs)
        # 2. Aggragate and discretize
        input_value_pairs = self.aggregate(strength)
     
        # 3. Defuzzify
        crisp_output= self.defuzzify(input_value_pairs)
        return crisp_output

    def aggregate(self, firing_strengths):
        # give firing strength to range of outputvalues
        list = []   
        for x in range(100, 23899):
            if x<=1000:
                list.append((firing_strengths['ultra cheap'],x))
    
            elif x>=1000 and x<3000:
                list.append((firing_strengths['bit cheap'],x))

            elif x>=3000 and x<5000:
                list.append((firing_strengths['cheap'], x))
            
            elif x>=5000 and x<10000:
                list.append((firing_strengths['medium'], x))
                
            elif x>=10000 and x<50000:
                list.append((firing_strengths['bit expensive'],x))

            elif x>=50000 and x<110000:
                list.append((firing_strengths['expensive'], x))
            
            elif x>=110000:
                list.append((firing_strengths['ultra expensive'], x))
        return list

    def defuzzify(self, input_value_pairs):
        # take largest value of peak
        if self.defuzzification == 'lom':
            value  = max(input_value_pairs) 
            crisp_value = value[1]
        # take smallest value of peak    
        if self.defuzzification == 'som':
            value  = max(input_value_pairs)
            for val in input_value_pairs:
                if val[0] == value[0]:
                    crisp_value = val[1]
                    break;
        return crisp_value

In [376]:
# Input previous output with the brand of car and go to defuzzify
datapoint = [2, q]
inputs= [brand, allPrices]
rulebase = Rulebase(rules_5)

# Return crisp value with largest of maximum, other methods can give better results.
thinker = Reasoner(rulebase, inputs, output, 201, "lom")
print("Price of car with largest of maxium:",round(thinker.inference(datapoint)), "euro")


Price of car with largest of maxium: 4999 euro
